# Part 4: Comparative Analysis of Pokémon Distribution by Primary Type and Generation

In [13]:
# 1. Grouped Bar Chart
import pandas as pd
import plotly.graph_objects as go

# Load the dataset
df = pd.read_pickle("/content/sample_data/cleaned_pokemon_data.pkl")

# Group by 'Generation' and 'Primary Type' and count occurrences
type_counts = df.groupby(["Generation", "Primary Type"]).size().unstack(fill_value=0)

# Color mapping for Pokémon types
type_colors = {
    'Grass': '#78C850', 'Fire': '#F08030', 'Water': '#6890F0', 'Bug': '#A8B820',
    'Normal': '#A8A878', 'Poison': '#A040A0', 'Electric': '#F8D030', 'Ground': '#E0C068',
    'Fairy': '#EE99AC', 'Fighting': '#C03028', 'Flying': '#A890F0', 'Psychic': '#F85888',
    'Rock': '#B8A038', 'Ghost': '#705898', 'Ice': '#98D8D8', 'Dragon': '#7038F8',
    'Dark': '#705848', 'Steel': '#B8B8D0'
}

# Create grouped bar chart
fig = go.Figure()

# Add a bar trace for each primary type
for i, ptype in enumerate(type_counts.columns):
    showlegend = True  # Show legend for the first bar of each type
    for gen in type_counts.index:
        count = type_counts.loc[gen, ptype]
        fig.add_trace(go.Bar(
            x=[gen],
            y=[count],
            name=ptype,
            marker={'color': type_colors.get(ptype)},
            width=0.05,
            hoverinfo='text',
            legendgroup=ptype,
            showlegend=showlegend,
        ))
        showlegend = False  # Hide legend for subsequent bars of this type


# Add dashed vertical lines to separate generations
for i, gen in enumerate(type_counts.index, start=1):
    fig.add_shape(
        dict(
            type="line",
            x0=i - 0.5,
            x1=i - 0.5,
            y0=0,
            y1=1,
            yref="paper",
            line=dict(color="Black", dash="dash", width=1)
        )
    )

# Layout settings
fig.update_layout(
    barmode='group',
    title="Pokémon Primary Type Distribution by Generation",
    xaxis_title="Generation",
    yaxis_title="Count",
    height=600,
    width=900,
    xaxis=dict(tickmode='linear'),
)

fig.show()


In [15]:
# 2. Stacked Bar Chart
fig = go.Figure()

for i, ptype in enumerate(type_counts.columns):
    showlegend = True
    for gen in type_counts.index:
        count = type_counts.loc[gen, ptype]
        fig.add_trace(go.Bar(
            x=[gen],
            y=[count],
            name=ptype,
            marker={'color': type_colors.get(ptype)},
            hovertext=f"{ptype}:{count}",
            hoverinfo='text',
            legendgroup=ptype,
            showlegend=showlegend,
        ))
        showlegend = False


# Layout settings
fig.update_layout(
    barmode='stack',
    title="Pokémon Primary Type Distribution by Generation",
    xaxis_title="Generation",
    yaxis_title="Primary Type Count",
    height=1200,
    width=1200,
    legend=dict(tracegroupgap=3)
)

fig.show()


In [19]:
# 3. Matrix Bar Chart
fig = go.Figure()

base_offset = 20  # Increment base by this value for each type
current_base = 0

for ptype in type_counts.columns:
    fig.add_trace(go.Bar(
        x=type_counts.index,
        y=type_counts[ptype],
        name=ptype,
        base=current_base,  # Use the current base for alignment
        marker_color=type_colors.get(ptype, '#CCCCCC'),
        opacity=0.8,  # Allow overlap visibility
        legendgroup=ptype
    ))
    current_base += base_offset  # Increment base for the next type

# Layout settings
fig.update_layout(
    barmode='stack',
    title="Pokémon Primary Type Distribution by Generation (Matrix Bar Chart)",
    xaxis_title="Generation",
    yaxis_showticklabels=False,  # Hide y-axis ticks
    height=1200,
    width=900,
)

fig.show()


In [22]:
# 4. Adding Negative Y-Axis for Totals

fig = go.Figure()

# Total Pokémon count per generation
total_counts = type_counts.sum(axis=1)

# Base value for matrix alignment
base_values = {ptype: i * 20 for i, ptype in enumerate(type_counts.columns)}

# Add bars for each primary type
for ptype, base in base_values.items():
    showlegend =True
    for gen in type_counts.index:
        fig.add_trace(go.Bar(
            x=[gen],
            y=[type_counts.loc[gen, ptype]],
            base=base,
            name=ptype,
            marker={'color': type_colors.get(ptype, '#CCCCCC')},
            hoverinfo='text',
            text=f'{ptype}: {type_counts.loc[gen, ptype]}',
            legendgroup=ptype,
            showlegend=showlegend
        ))
        showlegend = False

# Add bars for total counts in the negative y-axis
for gen in total_counts.index:
    showlegend = True
    fig.add_trace(go.Bar(
        x=[gen],
        y=[-total_counts[gen]],
        name="Total",
        marker={'color': '#333333'},
        hoverinfo='text',
        text=f'Total: {total_counts[gen]}',
        legendgroup="Total",
        showlegend=showlegend
    ))
    showlegend = False

# Layout settings
fig.update_layout(
    barmode='stack',
    title="Pokémon Primary Type Distribution by Generation",
    xaxis_title="Generation #",
    yaxis_title="Count",
    height=800,
    width=1000,
    xaxis=dict(tickmode='linear'),
    yaxis=dict(showticklabels=False),  # Hide y-axis ticks for clarity
)

fig.show()